# 4-Shadow Detector (OpenCV)

This notebook extracts shadow direction from the image and estimates an **observed sun azimuth**.

Pipeline:
1. Load image
2. Detect shadow mask (HSV threshold)
3. Estimate dominant shadow direction using PCA
4. Compute observed sun azimuth = shadow_angle + 180°
5. Save outputs:
   - shadow report JSON
   - shadow mask image
   - shadow overlay visualization


In [ ]:
!pip -q install opencv-python numpy matplotlib
import os, json, math
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
PROJECT_ROOT = "/content/orbitcheck"

IMG_DIR = os.path.join(PROJECT_ROOT, "data/raw/images")
META_DIR = os.path.join(PROJECT_ROOT, "data/raw/metadata")

REPORT_DIR = os.path.join(PROJECT_ROOT, "data/outputs/reports")
VIS_DIR = os.path.join(PROJECT_ROOT, "data/outputs/visualizations")

os.makedirs(REPORT_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

print("IMG_DIR:", IMG_DIR)
print("META_DIR:", META_DIR)
print("REPORT_DIR:", REPORT_DIR)
print("VIS_DIR:", VIS_DIR)

meta_files = [f for f in os.listdir(META_DIR) if f.endswith(".json")]
if len(meta_files) == 0:
    raise ValueError("No metadata JSON found. Run Notebook 01 first.")

print("\nAvailable metadata files:")
for f in meta_files:
    print(" -", f)

meta_filename = input("\nEnter metadata filename (example: sample1.json): ").strip()
meta_path = os.path.join(META_DIR, meta_filename)

with open(meta_path, "r") as f:
    meta = json.load(f)

image_filename = meta["image_filename"]
image_path = os.path.join(IMG_DIR, image_filename)

if not os.path.exists(image_path):
    raise FileNotFoundError(f"Image not found at {image_path}. Upload it again in Notebook 01.")

print("\n Loaded metadata for image:", meta["image_id"])
print("Image path:", image_path)


In [ ]:
img_bgr = cv2.imread(image_path)
if img_bgr is None:
    raise ValueError("Could not load image. Try JPG/PNG.")

img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(8,8))
plt.imshow(img_rgb)
plt.title("Input Image")
plt.axis("off")
plt.show()


In [ ]:
def detect_shadow_mask(img_bgr):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    # Shadows: low brightness + low/moderate saturation
    mask = cv2.inRange(hsv, (0, 0, 0), (180, 100, 110))

    # Morphological cleanup
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    return mask

shadow_mask = detect_shadow_mask(img_bgr)

plt.figure(figsize=(8,8))
plt.imshow(shadow_mask, cmap="gray")
plt.title("Shadow Mask (Detected)")
plt.axis("off")
plt.show()

print("Shadow pixels:", int(np.sum(shadow_mask > 0)))


In [ ]:
def estimate_shadow_direction_pca(shadow_mask, min_pixels=800):
    ys, xs = np.where(shadow_mask > 0)

    if len(xs) < min_pixels:
        return None, 0.0

    coords = np.vstack((xs, ys)).T.astype(np.float32)

    # PCA: compute principal axis
    mean = np.mean(coords, axis=0)
    centered = coords - mean
    cov = np.cov(centered.T)

    eigvals, eigvecs = np.linalg.eig(cov)

    principal = eigvecs[:, np.argmax(eigvals)]
    dx, dy = principal

    # Image axis: x right, y down
    angle_rad = math.atan2(dy, dx)
    angle_deg = (math.degrees(angle_rad) + 360) % 360

    # confidence: more shadow pixels -> higher confidence
    confidence = min(1.0, len(xs) / 60000)

    return float(angle_deg), float(confidence)

shadow_angle, shadow_conf = estimate_shadow_direction_pca(shadow_mask)

print("\n Shadow Direction (image axis):", shadow_angle)
print("Shadow Confidence:", shadow_conf)


In [ ]:
def shadow_to_sun_azimuth(shadow_angle_deg):
    # Sun direction is opposite to shadow direction
    return (shadow_angle_deg + 180) % 360

if shadow_angle is None:
    observed_sun_az = None
    print("Not enough shadows detected for a reliable direction.")
else:
    observed_sun_az = shadow_to_sun_azimuth(shadow_angle)
    print("\n Observed Sun Azimuth (from shadow):", observed_sun_az)


In [ ]:
def draw_arrow(img_rgb, angle_deg, color=(255, 0, 0), label=""):
    h, w, _ = img_rgb.shape
    cx, cy = w // 2, h // 2
    length = min(h, w) // 4

    angle_rad = math.radians(angle_deg)
    x2 = int(cx + length * math.cos(angle_rad))
    y2 = int(cy + length * math.sin(angle_rad))

    out = img_rgb.copy()
    cv2.arrowedLine(out, (cx, cy), (x2, y2), color, 6, tipLength=0.12)

    if label:
        cv2.putText(out, label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
    return out

overlay = img_rgb.copy()

if shadow_angle is not None:
    overlay = draw_arrow(overlay, shadow_angle, color=(255, 255, 0), label=f"Shadow Dir: {shadow_angle:.1f}°")

if observed_sun_az is not None:
    overlay = draw_arrow(overlay, observed_sun_az, color=(0, 255, 0), label=f"Observed Sun Az: {observed_sun_az:.1f}°")

plt.figure(figsize=(10,10))
plt.imshow(overlay)
plt.title("Shadow + Observed Sun Azimuth")
plt.axis("off")
plt.show()


In [ ]:
image_id = meta["image_id"]

mask_path = os.path.join(VIS_DIR, f"{image_id}_shadow_mask.png")
overlay_path = os.path.join(VIS_DIR, f"{image_id}_shadow_overlay.png")
report_path = os.path.join(REPORT_DIR, f"{image_id}_shadow_report.json")

cv2.imwrite(mask_path, shadow_mask)
cv2.imwrite(overlay_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

shadow_report = {
    "image_id": image_id,
    "image_filename": image_filename,
    "shadow_pixel_count": int(np.sum(shadow_mask > 0)),
    "shadow_direction_deg_image_axis": shadow_angle,
    "shadow_confidence": shadow_conf,
    "observed_sun_azimuth_deg": observed_sun_az,
    "notes": "Observed sun azimuth assumes north-up image orientation."
}

with open(report_path, "w") as f:
    json.dump(shadow_report, f, indent=2)

print("\n Saved outputs:")
print("Shadow mask:", mask_path)
print("Overlay:", overlay_path)
print("Shadow report JSON:", report_path)

print("\nReport Preview:")
print(json.dumps(shadow_report, indent=2))
